# SQL Documentation Agent - Interactive Demo

This notebook demonstrates the capabilities of the Smol-SQL Agents Suite for autonomous database documentation generation with semantic search capabilities.

## Features Demonstrated:
- 🔍 Database schema discovery
- 📝 AI-powered documentation generation
- 🔍 Vector indexing with ChromaDB
- 🔍 Semantic search capabilities
- ⚡ Batch processing for efficiency
- 💾 State persistence and resume capability
- 📊 Cost estimation
- 🔄 Index management

## 1. Setup and Environment Configuration

In [1]:
%pip install -r smol-sql-agents/requirements.txt

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Add the project root to Python path
project_root = Path.cwd()
sys.path.append(str(project_root) + "/smol-sql-agents")

# Load environment variables
load_dotenv()

# Import the framework components
from src.database.inspector import DatabaseInspector
from src.agents.core import PersistentDocumentationAgent
from src.output.formatters import DocumentationFormatter
from src.agents.batch_manager import BatchIndexingManager
from src.vector.search import search_table_documentation, search_relationship_documentation

print("✅ Framework imports successful")
print(f"📁 Project root: {project_root}")

## 2. Environment Configuration Check

In [ ]:
# Check environment configuration
print("🔧 Environment Configuration Check")
print("=" * 50)

# Check required environment variables
required_vars = [
    'OPENAI_API_KEY',
    'DATABASE_URL'
]

for var in required_vars:
    value = os.getenv(var)
    if value:
        # Mask sensitive values
        if 'API_KEY' in var:
            display_value = value[:8] + "..." + value[-4:] if len(value) > 12 else "***"
        else:
            display_value = value
        print(f"✅ {var}: {display_value}")
    else:
        print(f"❌ {var}: Not set")

# Check optional environment variables
optional_vars = [
    'LOG_LEVEL',
    'EMBEDDING_BATCH_SIZE',
    'EMBEDDING_MAX_RETRIES'
]

print("\n📋 Optional Configuration:")
for var in optional_vars:
    value = os.getenv(var, "Not set (using default)")
    print(f"  {var}: {value}")

## 3. Database Connection and Schema Discovery

In [ ]:
# Initialize database inspector
print("🔍 Database Schema Discovery")
print("=" * 50)

try:
    inspector = DatabaseInspector()
    
    # Discover all tables
    tables = inspector.get_all_table_names()
    print(f"📊 Found {len(tables)} tables:")
    for i, table in enumerate(tables[:10], 1):  # Show first 10
        print(f"  {i}. {table}")
    if len(tables) > 10:
        print(f"  ... and {len(tables) - 10} more tables")
    
    # Discover relationships
    relationships = inspector.get_all_foreign_key_relationships()
    print(f"\n🔗 Found {len(relationships)} relationships:")
    for i, rel in enumerate(relationships[:5], 1):  # Show first 5
        print(f"  {i}. {rel['constrained_table']} -> {rel['referred_table']}")
    if len(relationships) > 5:
        print(f"  ... and {len(relationships) - 5} more relationships")
    
    # Show sample table schema
    if tables:
        sample_table = tables[0]
        print(f"\n📋 Sample schema for '{sample_table}':")
        schema = inspector.get_table_schema(sample_table)
        print(f"  Columns: {len(schema.get('columns', []))}")
        for col in schema.get('columns', [])[:3]:  # Show first 3 columns
            print(f"    - {col.get('name')}: {col.get('type')}")
        if len(schema.get('columns', [])) > 3:
            print(f"    ... and {len(schema.get('columns', [])) - 3} more columns")
    
    print("\n✅ Database connection and schema discovery successful!")
    
except Exception as e:
    print(f"❌ Database connection failed: {e}")
    print("\n💡 Make sure your DATABASE_URL is correctly configured.")

## 4. Agent Initialization and Vector Indexing Status

In [ ]:
# Initialize the main agent
print("🤖 Agent Initialization")
print("=" * 50)

try:
    agent = PersistentDocumentationAgent()
    
    print(f"✅ Agent initialized successfully")
    print(f"📊 Vector indexing available: {agent.vector_indexing_available}")
    print(f"🔍 Indexer agent available: {'Yes' if agent.indexer_agent else 'No'}")
    print(f"💾 Database store available: {'Yes' if agent.store else 'No'}")
    print(f"🧠 LLM model available: {'Yes' if agent.llm_model else 'No'}")
    
    if agent.vector_indexing_available:
        print("\n🎉 Vector indexing is available!")
        print("You can use advanced features like:")
        print("  - Semantic search")
        print("  - Batch processing")
        print("  - Cost estimation")
    else:
        print("\n⚠️  Vector indexing is not available.")
        print("Basic documentation generation will still work.")
    
except Exception as e:
    print(f"❌ Agent initialization failed: {e}")
    print("\n💡 Check your OpenAI API key and database connection.")

## 5. Documentation Generation Demo

In [ ]:
# Demo: Process a single table
print("📝 Documentation Generation Demo")
print("=" * 50)

if 'inspector' in locals() and 'agent' in locals():
    try:
        # Get a sample table
        tables = inspector.get_all_table_names()
        if tables:
            sample_table = tables[0]
            print(f"🎯 Processing table: {sample_table}")
            
            # Process the table documentation
            agent.process_table_documentation(sample_table)
            
            print(f"✅ Successfully processed table: {sample_table}")
            print("\n📋 Generated documentation includes:")
            print("  - Business purpose inference")
            print("  - Schema analysis")
            print("  - Column descriptions")
            
            # Check if it was indexed (if vector indexing is available)
            if agent.vector_indexing_available:
                print("  - Vector indexing (for semantic search)")
            
        else:
            print("❌ No tables found in database")
            
    except Exception as e:
        print(f"❌ Table processing failed: {e}")
else:
    print("❌ Agent or inspector not initialized")

## 6. Vector Indexing and Search Demo

In [ ]:
# Demo: Vector indexing and semantic search
print("🔍 Vector Indexing and Search Demo")
print("=" * 50)

if 'agent' in locals() and agent.vector_indexing_available:
    try:
        # Check indexing status
        print("📊 Checking vector indexing status...")
        
        # Get indexing statistics
        batch_manager = BatchIndexingManager(agent.indexer_agent)
        stats = batch_manager.get_processing_stats(agent.store)
        
        print(f"\n📈 Indexing Statistics:")
        print(f"  Pending tables: {stats['pending_tables']}")
        print(f"  Pending relationships: {stats['pending_relationships']}")
        print(f"  Total pending: {stats['total_pending']}")
        print(f"  Batch size: {stats['batch_size']}")
        
        # Perform a sample search
        print("\n🔍 Performing sample search...")
        search_query = "user"  # Simple search term
        
        results = agent.indexer_agent.search_documentation(search_query, "all")
        
        if results:
            total_results = results.get("total_results", 0)
            print(f"✅ Search completed: {total_results} results found")
            
            # Display results
            if results.get("tables"):
                print(f"\n📊 Tables ({len(results['tables'])} results):")
                for i, table in enumerate(results["tables"][:3], 1):  # Show first 3
                    table_name = table.get('content', {}).get('name', 'Unknown')
                    similarity = table.get('score', 0)
                    print(f"  {i}. {table_name} (similarity: {similarity:.3f})")
            
            if results.get("relationships"):
                print(f"\n🔗 Relationships ({len(results['relationships'])} results):")
                for i, rel in enumerate(results["relationships"][:3], 1):  # Show first 3
                    rel_name = rel.get('content', {}).get('name', 'Unknown')
                    similarity = rel.get('score', 0)
                    print(f"  {i}. {rel_name} (similarity: {similarity:.3f})")
        else:
            print("❌ Search returned no results")
            
    except Exception as e:
        print(f"❌ Vector indexing demo failed: {e}")
else:
    print("⚠️  Vector indexing is not available")
    print("This demo requires vector indexing to be enabled.")

## 7. Entity Recognition Demo

In [ ]:
# Entity recognition demo
print("🔍 Entity Recognition Demo")
print("=" * 50)

if 'agent' in locals() and agent.vector_indexing_available:
    try:
        # Import entity recognition agent
        from src.agents.entity_recognition import EntityRecognitionAgent
        
        # Initialize entity recognition agent
        entity_agent = EntityRecognitionAgent(agent.indexer_agent)
        print("✅ Entity recognition agent initialized")
        
        # Sample queries for entity recognition
        sample_queries = [
            "customers information"
        ]
        
        print("\n🔍 Available sample queries:")
        for i, query in enumerate(sample_queries, 1):
            print(f"  {i}. '{query}'")
        
        print("\n💡 Running entity recognition for all sample queries...\n")
        
        for idx, demo_query in enumerate(sample_queries, 1):
            print("=" * 50)
            print(f"🎯 Entity Recognition {idx}: '{demo_query}'")
            try:
                results = entity_agent.recognize_entities(demo_query)
                
                if results and results.get("success"):
                    confidence = results.get("confidence", 0.0)
                    analysis = results.get("analysis", "No analysis available")
                    applicable_entities = results.get("applicable_entities", [])
                    
                    print(f"✅ Success! Confidence Score: {confidence:.2f}")
                    print(f"Analysis: {analysis}")
                    
                    if applicable_entities:
                        print(f"\n📊 Applicable Entities ({len(applicable_entities)} found):")
                        for i, entity in enumerate(applicable_entities, 1):
                            table_name = entity.get("table_name", "Unknown")
                            relevance_score = entity.get("relevance_score", 0.0)
                            business_purpose = entity.get("business_purpose", "")
                            recommendation = entity.get("recommendation", "")
                            
                            print(f"\n  {i}. {table_name}")
                            print(f"     Relevance: {relevance_score:.3f}")
                            print(f"     Purpose: {business_purpose}")
                            if recommendation:
                                print(f"     Recommendation: {recommendation}")
                    else:
                        print("\n📊 No applicable entities found.")
                        
                    # Show recommendations if available
                    recommendations = results.get("recommendations", [])
                    if recommendations:
                        print(f"\n💡 Entity Recommendations ({len(recommendations)} items):")
                        for i, rec in enumerate(recommendations, 1):
                            table_name = rec.get("table_name", "Unknown")
                            relevance_score = rec.get("relevance_score", 0.0)
                            business_purpose = rec.get("business_purpose", "")
                            print(f"  {i}. {table_name} (relevance: {relevance_score:.3f})")
                            if business_purpose:
                                print(f"     Purpose: {business_purpose}")
                else:
                    error = results.get("error", "Unknown error") if results else "No results returned"
                    print(f"❌ Entity recognition failed: {error}")
                    if results and results.get("details"):
                        print(f"Details: {results['details']}")
                        
            except Exception as e:
                print(f"❌ Entity recognition failed: {e}")
            print("=" * 50 + "\n")
            
        # Quick entity lookup demo
        print("\n" + "=" * 60)
        print("🚀 Quick Entity Lookup Demo")
        print("=" * 60)
        
        quick_query = "customer information"
        print(f"Query: '{quick_query}'")
        
        try:
            table_names = entity_agent.quick_entity_lookup(quick_query, threshold=0.3)
            
            if table_names:
                print(f"✅ Found {len(table_names)} relevant tables:")
                for i, table_name in enumerate(table_names, 1):
                    print(f"  {i}. {table_name}")
            else:
                print("❌ No relevant tables found above threshold.")
                
        except Exception as e:
            print(f"❌ Quick lookup failed: {e}")
            
    except Exception as e:
        print(f"❌ Entity recognition demo failed: {e}")
else:
    print("⚠️  Entity recognition requires vector indexing")
    print("Enable vector indexing to use entity recognition features.")